In [1]:
import plotly 
plotly.tools.set_credentials_file(username='green.jo', api_key='oSu2L4HiKfd37uDlaDT0')

In [2]:
from NOAAStations import TidalStation
from DeviceModels import Turbine, calculate_power
from Calculator import maintenance, operation, installation

import pandas as pd
import os
from ipywidgets import widgets, interact, fixed
from IPython.display import display
%matplotlib inline
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.pylabtools import figsize
import scipy
import scipy.interpolate
from contextlib import redirect_stdout
figsize(12, 10)
sbn.set_context("paper", font_scale=1)
sbn.set_style("whitegrid")


from collections import namedtuple


### Testing for the maintenance monte carlo simulation

In [3]:
def harmonicConstituentModel(time, *hm):
    assert len(hm) % 3 == 0
    velocity = 0 
    for i in range(len(hm)//3):
        velocity += hm[3*i]*np.cos((hm[3*i+1] * time + hm[3*i+2])*np.pi/180.)
    return velocity

def calculate_Installation(installations):
    return sum([i.capEx for i in installations])

def calculate_ops(ops, time):
    return(sum([o.annualCost*time for o in ops]))

def richardsCurve(Velocity,B,M,g):
    return 1200*(1+.1835*np.exp(-1*B*(Velocity-M)))**(-1/g)

def interpolate(value, from_a, from_b, to_a, to_b):
    return to_a +((to_a-to_b)/(from_a-from_b))*(value - from_a)        

def LevelizedCostofElectricity(HM,
                               number_of_turbines,
                               lifetime, 
                               K, Q, B, M, g,
                               emergency_maintentance,
                               installation,
                               operations,
                               power_array = None,
                               num = 500):
    
    '''
    This function will calculated the levelized cost of electricity given the parameters for maintenance, power generation, installation
    and lifetime
    station_id will determine the location due to the necessity to use harmonic constituents for the calculations
    grid_location is where the connections will be made
    cap_ex are the capital expenditures for the cost of the turbine and fixtures
    this function was written with a sensitivity analysis in mind
    '''
    
    MCT = Turbine(K, Q, B, M, g)
        
    if power_array is None:
        power_array , time_array = calculate_power(HM, 
                                                     MCT, 
                                                     0, 
                                                     0, 
                                                     lifetime*24*3600*365.25, 
                                                     ) # everything else is in years, need to convert to seconds for int
        time_array = time_array/(24.*3600.*365.25)
    else: 
        time_array = np.linspace(0, lifetime, len(power_array))
    ###
    # The following code is used to run the monte carlo simulation with feedback to the power generation functions
    # where the downtime requires the turbine to no longer generate an output
    ###

    power_array *= .95 #to account for voltage drop across cable
    maintenance_costs = np.zeros(num)
    power_losses = np.zeros_like(maintenance_costs)
    #time to run the simulation
    
    for i in range(num):            
        end_loop = False
        time_tracker = 0.
        power_loss = 0.
        maintenance_cost = 0.
        
        for turbine in range(number_of_turbines):
            while not end_loop:
                maintenance_event, uptime = maintenance.monteCarlo(emergency_maintentance)
                end_time = time_tracker + uptime
                maintenance_cost += maintenance_event.event_cost
                time_tracker += uptime + maintenance_event.downtime.total_seconds()/(24*3600*365.25)
                if end_time >= lifetime or time_tracker >= lifetime:
                    break
                start_index = np.searchsorted(time_array, time_tracker)
                end_index = np.searchsorted(time_array, end_time)
                energy_2 = interpolate(time_tracker, time_array[start_index-1], time_array[start_index], 
                                       power_array[start_index-1], power_array[start_index])
                energy_1 = interpolate(end_time, time_array[end_index-1], time_array[end_index], 
                                       power_array[end_index-1], power_array[end_index])
                power_loss += energy_2 - energy_1

        power_losses[i] = power_loss
        maintenance_costs[i] = maintenance_cost

    installation_cost = calculate_Installation(installation)
    planned_maintenance = .05 * installation_cost
    ops_cost = calculate_ops(operations, lifetime)
    # Process the final costs and return the levelized cost
    total_cost = np.mean(maintenance_costs) + installation_cost + ops_cost + planned_maintenance
    total_power = (power_array[-1]*number_of_turbines - np.mean(power_losses))/3600 #to kWhr!!
    print('Ideal power output = {} MWhr '.format(power_array[-1]/(1000*3600)))
    print('Estimated total power loss - {} MJ, sigma = {}'.format(np.mean(power_losses)/1000, np.std(power_losses)/1000))
    print('Estimated total maintenance cost - $ {}, sigma = $ {}'.format(np.mean(maintenance_costs), np.std(maintenance_costs)))
    print('Estimated installation cost - $ {}'.format(installation_cost))
    print('Estimated operations cost - $ {}'.format(ops_cost))
    return total_cost/total_power, power_array

In [4]:
Maintenance_Rate = namedtuple('Parameter', 'partname minimal_rate midlevel_rate severe_rate minimal_cost midlevel_cost severe_cost number labor')
CapitalInstallation = namedtuple('Parameter', 'name costPerDay timePerTurbine time numberOfTurbines scalePerTurbine')

## Heli Rate 2000-5000
## Heli Speed = 130-145
## 1014
## 31


emergency_maintenance = [
    Maintenance_Rate('Blade', 0.042, 0.0273, 0.00007, 1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Others', 0.03, 0.0299, 0.00006, 1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Gear Box',0.2125, 0.0325, 0.0005, 1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Electricity Generator', 0.065, 0.0545, 0.0065, 1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Shaft', 0.002, 0.007, .001, 1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Brake', 0.0153, 0.0325, 0.0025,1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Cable', 0.225, 0.09247, 0.000002,1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.),
    Maintenance_Rate('Control system', 0.1, 0.1, 0.0001,1014.*24.*1, 1014*24.*4, 3500*24.*5, 1., 3*40.)
]


emergency_events = [maintenance.EmergencyMaintenance(
            e.minimal_rate, 
            e.midlevel_rate, 
            e.severe_rate,
            e.minimal_cost, 
            e.midlevel_cost, 
            e.severe_cost,
            number = e.number, 
            labor = e.labor, 
            partname = e.partname)
            for e in emergency_maintenance]

ops = [
    operation.OperationsCrew('Site Manager', 1, 114750),
    operation.OperationsCrew('Admin Asst', 2, 94500),
    operation.OperationsCrew('Sr. Tech', 3, 126360/3),
    operation.OperationsCrew('Jr. Tech', 6, 219024/6),
]

HM = []
with open('HM-COI0301-38.txt','r') as myFile:
    for line in myFile:
        amplitude, speed, phase  = line.split(',')
        HM.append(float(amplitude))
        HM.append(float(speed))
        HM.append(float(phase))

HM = tuple(HM)

# MCT = Turbine(1200., 0.1835, 3.55361367,  2.30706792,  1.05659521)
# Sagamore = TidalStation(8447173)
# results, times = calculate_power(Sagamore, MCT, 0, 0, 365*24*3600, 9.8, 3)
# t = np.arange(0, 50)
# graph2 = harmonicConstituentModel(t, *HM)

# plt.plot(t, graph2, label='Least Squares Fit')
# plt.legend(loc='best')
# plt.xlabel('Time (hours)')
# plt.ylabel('Velocity (m/s)')
# plt.savefig('TidalCurrentHM.png', format='png', transparent=True, bbox_inches='tight')


In [5]:
lifetime = 20.
LCOE = []
for number_of_turbines in [1,2,5,10,50,100]:
    print('Starting calculations for {} turbines'.format(number_of_turbines))

    Capital_Installations = [
    CapitalInstallation('Pile Installation, Mobilize Vessel', 111000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Transport', 167000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Drive Piles', 164000., .3, 'n/a', number_of_turbines, True),
    CapitalInstallation('Pile Installation, transport home', 167000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Demobilize', 110000., 'n/a', 3, number_of_turbines, False),
    CapitalInstallation('Gunderboom Sound Barrier', 4500000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Frame for Barrier',50000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Mob/Demob Sound Barrier', 70000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cable transport to site',45000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cables install to device',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable to pile',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable Splicing',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable Fairleading',75000., 'n/a', 5, number_of_turbines, False),
    CapitalInstallation('Cable through HDD', 75000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Cable Burial', 75000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Cable Testing and Commissioning',63000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Cable Transport Home', 45000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cable - Demobilization', 46000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Device Installation, Mobilize Vessel', 74000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Device Installation, Transport to site', 79000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Device Installation, install',  106000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Device Installation, Secure Cables', 106000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Device Installation, Fairleading Cables',  106000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Device Installation, Transport Home', 87000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('FERC Filing Fee', 91000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Device', 3000000., 1, 'n/a', number_of_turbines, True)]

    installations = [installation.CapitalInstallation(i.name, 
                                                      i.time, 
                                                      i.timePerTurbine, 
                                                      i.costPerDay, 
                                                      i.numberOfTurbines, 
                                                      i.scalePerTurbine)
                                                      for i in Capital_Installations ]
    
    ops.append(operation.OperationsCrew('Lease', 1, 839*number_of_turbines/3))
    
    if number_of_turbines == 1:
        result, power_array = LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 1200., 0.1835, 3.55361367,  2.30706792,  1.05659521,
                               emergency_events, installations, ops)
    else:
        result, power_array = LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 1200., 0.1835, 3.55361367,  2.30706792,  1.05659521,
                               emergency_events, installations, ops, power_array = power_array)        
    
    LCOE.append(result)
    print('LCOE for {} turbine(s) was {}'.format(number_of_turbines, LCOE[-1]))
    print('-'*80)

print('1. SeaGen - {}'.format(LCOE))




Starting calculations for 1 turbines
Ideal power output = [ 187535.97104136] MWhr 
Estimated total power loss - 8934673.3280912 MJ, sigma = 2132009.172662341
Estimated total maintenance cost - $ 1498408.512, sigma = $ 396853.83654812997
Estimated installation cost - $ 11353700.0
Estimated operations cost - $ 12988273.333333334
LCOE for 1 turbine(s) was [ 0.14270456]
--------------------------------------------------------------------------------
Starting calculations for 2 turbines
Ideal power output = [ 178159.17248929] MWhr 
Estimated total power loss - 8452114.084002184 MJ, sigma = 2122836.1889870833
Estimated total maintenance cost - $ 1562538.624, sigma = $ 424453.2332388795
Estimated installation cost - $ 14621400.0
Estimated operations cost - $ 12999460.0
LCOE for 2 turbine(s) was [ 0.08451118]
--------------------------------------------------------------------------------
Starting calculations for 5 turbines
Ideal power output = [ 169251.21386483] MWhr 
Estimated total power l

In [6]:
lifetime = 20.
LCOE_gen4wave = []
for number_of_turbines in [1,2,5,10,50,100]:
    print('Starting calculations for {} turbines'.format(number_of_turbines))

    Capital_Installations = [
    CapitalInstallation('Pile Installation, Mobilize Vessel', 111000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Transport', 167000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Drive Piles', 164000., .3, 'n/a', number_of_turbines, True),
    CapitalInstallation('Pile Installation, transport home', 167000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Pile Installation, Demobilize', 110000., 'n/a', 3, number_of_turbines, False),
    CapitalInstallation('Gunderboom Sound Barrier', 4500000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Frame for Barrier',50000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Mob/Demob Sound Barrier', 70000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cable transport to site',45000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cables install to device',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable to pile',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable Splicing',75000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Cable Fairleading',75000., 'n/a', 5, number_of_turbines, False),
    CapitalInstallation('Cable through HDD', 75000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Cable Burial', 75000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Cable Testing and Commissioning',63000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Cable Transport Home', 45000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Cable - Demobilization', 46000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Device Installation, Mobilize Vessel', 74000., 'n/a', 4, number_of_turbines, False),
    CapitalInstallation('Device Installation, Transport to site', 79000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Device Installation, install',  106000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Device Installation, Secure Cables', 106000., .5, 'n/a', number_of_turbines, True),
    CapitalInstallation('Device Installation, Fairleading Cables',  106000., 'n/a', 2, number_of_turbines, False),
    CapitalInstallation('Device Installation, Transport Home', 87000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('FERC Filing Fee', 91000., 'n/a', 1, number_of_turbines, False),
    CapitalInstallation('Device', 3000000., 1, 'n/a', number_of_turbines, True),
    CapitalInstallation('Additional Device Fee', 9000000., 'n/a', 1, number_of_turbines, False)]

    installations = [installation.CapitalInstallation(i.name, 
                                                      i.time, 
                                                      i.timePerTurbine, 
                                                      i.costPerDay, 
                                                      i.numberOfTurbines, 
                                                      i.scalePerTurbine)
                                                      for i in Capital_Installations ]
    if number_of_turbines == 1:
        result, power_array = LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 1164.785 , 2.834 , 2.778 , 1.020 , 0.751,
                               emergency_events, installations, ops)
    else:
        result, power_array = LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 1164.785 , 2.834 , 2.778 , 1.020 , 0.751, 
                               emergency_events, installations, ops, power_array=power_array)      
    LCOE_gen4wave.append(result)
    
    print('LCOE for {} turbine(s) was {}'.format(number_of_turbines, LCOE_gen4wave[-1]))
print('2. Gen4Wave V7 - {}'.format(LCOE_gen4wave))

Starting calculations for 1 turbines
Ideal power output = [ 180648.4822635] MWhr 
Estimated total power loss - 8542788.833702302 MJ, sigma = 2055457.3092937882
Estimated total maintenance cost - $ 1489782.624, sigma = $ 408017.2506855398
Estimated installation cost - $ 20020700.0
Estimated operations cost - $ 13922360.0
LCOE for 1 turbine(s) was [ 0.20436841]
Starting calculations for 2 turbines
Ideal power output = [ 171616.05815032] MWhr 
Estimated total power loss - 7934461.931685165 MJ, sigma = 2060496.3820628359
Estimated total maintenance cost - $ 1522295.52, sigma = $ 420649.16259114747
Estimated installation cost - $ 23288400.0
Estimated operations cost - $ 13922360.0
LCOE for 2 turbine(s) was [ 0.11699175]
Starting calculations for 5 turbines
Ideal power output = [ 163035.25524281] MWhr 
Estimated total power loss - 7624409.225793967 MJ, sigma = 1837217.4474487805
Estimated total maintenance cost - $ 1723307.616, sigma = $ 414508.962892467
Estimated installation cost - $ 33091

In [7]:
lifetime = 20.
LCOE_RM4 = []
for number_of_turbines in [1,2,5,10,50,100]:
    print('Starting calculations for {} turbines'.format(number_of_turbines))
    def calculateCapital(num_turbs):
        return -30482*num_turbs**2 + 3e7*num_turbs +9e7
    Capital_Installations = [
    CapitalInstallation('Generalized', calculateCapital(number_of_turbines), 'n/a', 1, number_of_turbines, False)]

    installations = [installation.CapitalInstallation(i.name, 
                                                      i.time, 
                                                      i.timePerTurbine, 
                                                      i.costPerDay, 
                                                      i.numberOfTurbines, 
                                                      i.scalePerTurbine)
                                                      for i in Capital_Installations ]
   

    ops.append(operation.OperationsCrew('Lease', 1, 839*number_of_turbines*28800/100))


    if number_of_turbines == 1:
        result, power_array = (LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 3815.76, 0.83, 4.38412328, 1.32480294, 0.952668935,
                               emergency_events, installations, ops))
    else:
        result, power_array = LevelizedCostofElectricity(HM, number_of_turbines, lifetime, 3815.76, 0.83, 4.38412328, 1.32480294, 0.952668935,
                               emergency_events, installations, ops, power_array = power_array)       
    LCOE_RM4.append(result)
    
    print('LCOE for {} turbine(s) was {}'.format(number_of_turbines, LCOE_RM4[-1]))
print('3. RM4, Moored Glider, 4 axial-flow - {}'.format(LCOE_RM4))

Starting calculations for 1 turbines
Ideal power output = [ 621500.0860493] MWhr 
Estimated total power loss - 28742210.465823166 MJ, sigma = 7170061.315653093
Estimated total maintenance cost - $ 1451589.216, sigma = $ 409067.6415934672
Estimated installation cost - $ 119969518.0
Estimated operations cost - $ 18755000.0
LCOE for 1 turbine(s) was [ 0.23825711]
Starting calculations for 2 turbines
Ideal power output = [ 590425.08174683] MWhr 
Estimated total power loss - 27275882.55204691 MJ, sigma = 6410989.908423449
Estimated total maintenance cost - $ 1522867.2, sigma = $ 397095.3905644738
Estimated installation cost - $ 149878072.0
Estimated operations cost - $ 28420280.0
LCOE for 2 turbine(s) was [ 0.15965171]
Starting calculations for 5 turbines
Ideal power output = [ 560903.82765949] MWhr 
Estimated total power loss - 26241755.93629947 MJ, sigma = 6418309.68249136
Estimated total maintenance cost - $ 1748095.776, sigma = $ 436263.32305092737
Estimated installation cost - $ 239237

In [8]:
print(ops[-1].position)

Lease


In [17]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = [go.Bar(
    x = ['One','Two','Five','Ten','Fifty','One Hundred'],
    y = LCOE,
    marker = dict(color = '#104E8B'),
    name = 'SeaGen'),
    go.Bar(
    x = ['One','Two','Five','Ten','Fifty','One Hundred'],
    y = LCOE_gen4wave,
    marker = dict(color = '#8470FF'),
    name = 'Gen4wave V7'),
    go.Bar(
    x = ['One','Two','Five','Ten','Fifty','One Hundred'],
    y = LCOE_RM4,
    marker = dict(color = '#63B8FF'),
    name = 'Ocean Current Turbine')]

layout = go.Layout(
    xaxis = dict(title = 'Number of Turbines',
        titlefont = dict(
        size = 20,
        color = 'white'),
        tickfont=dict(
            size=16,
            color='white'
        )),
    yaxis = dict(title = 'US $ / kWhr',
        titlefont = dict(
        size = 20,
        color = 'white'),
        tickfont=dict(
            size=16,
            color='white'
        )),
    paper_bgcolor='transparent',
    plot_bgcolor='transparent',
    legend = dict(orientation="h",
        font = dict(color = 'white')))

fig = go.Figure(data = trace, layout=layout)
py.iplot(fig, filename='KnikArm')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~green.jo/0 or inside your plot.ly account where it is named 'KnikArm'
